# Data wrangling over excel files

---



> ## **A prior implementation**: descriptions are included

In [1]:
!pip install polars 
!pip install xlsx2csv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.6 MB 12.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import polars as pl
import pandas as pd
import xlrd as xls
import numpy as np

In [3]:
file = "/content/existencias-animales.xlsx"

In [40]:
get_ini = lambda x, xs: [index+1 for (name, pibot, index) in zip(xs, ["><"]+xs, range(len(xs))) if name == pibot]
get_indexes = lambda x, xs: [i for (y, i) in zip(xs, range(len(xs))) if x == y]

Obtaining indexes for sectoring data content

In [73]:
work_book = xls.open_workbook(file)
work_book = work_book.sheet_by_index(0)

column = 1

array_col = work_book.col_values(column)[1:]
ini = get_ini("", array_col)[-1]
fin = get_indexes("", array_col)[-1]

array_col = array_col[ini: fin]

print(array_col)

titles = array_col

['Total Nacional - Región', 'Total Nacional', 'Región de Arica y Parinacota', 'Región de Tarapacá ', 'Región de Antofagasta', 'Región de Atacama', 'Región de Coquimbo', 'Región de Valparaíso', 'Región Metropolitana de Santiago', "Región del Libertador General Bernardo O'Higgins", 'Región del Maule', 'Región de Ñuble', 'Región del Biobío ', 'Región de La Araucanía', 'Región de Los Ríos', 'Región de Los Lagos', 'Región de Aysén del General Carlos Ibáñez del Campo', 'Región de Magallanes y de la Antártica Chilena']


Selecting columns as arrays
Sizing data of the columns
Data normalization
Storing arrays into fields

This process will finish when there are no more columns with data

In [74]:
fields = np.array(array_col)

column=2
while True:
  try:
    array_col = work_book.col_values(column)[1:]
    array_col = array_col[ini:fin]
    column+=1
    ## Data normalization
    array_col = np.char.lower(array_col)
    array_col = np.char.replace(array_col, " ", "_")
    #print(array_col)
    fields = np.row_stack((fields, array_col))
  except:
    print("Ending column")
    break

3 21
Ending column


In [75]:
array_to_dataframe = np.transpose(fields)
titles = array_to_dataframe[0]
print(titles)


df_polar = pl.from_numpy(data=array_to_dataframe, columns=list(titles),)
df_polar = df_polar[1:]
print(df_polar)

df_polar.write_csv(file.split('.')[0]+'.csv')


['Total Nacional - Región' 'bovinos' 'ovinos' 'caprinos' 'porcinos'
 'caballares' 'mulares_y_burdéganos' 'asnales' 'llamas' 'alpacas'
 'huarizos' 'conejos' 'jabalíes' 'ciervos' 'gallinas/pollos_4_' 'pavos'
 'otras_aves_de_corral_5']
shape: (17, 17)
┌────────────┬───────────┬───────────┬──────────┬─────┬─────────┬────────────┬───────────┬────────────┐
│ Total      ┆ bovinos   ┆ ovinos    ┆ caprinos ┆ ... ┆ ciervos ┆ gallinas/p ┆ pavos     ┆ otras_aves │
│ Nacional - ┆ ---       ┆ ---       ┆ ---      ┆     ┆ ---     ┆ ollos_4_   ┆ ---       ┆ _de_corral │
│ Región     ┆ str       ┆ str       ┆ str      ┆     ┆ str     ┆ ---        ┆ str       ┆ _5         │
│ ---        ┆           ┆           ┆          ┆     ┆         ┆ str        ┆           ┆ ---        │
│ str        ┆           ┆           ┆          ┆     ┆         ┆            ┆           ┆ str        │
╞════════════╪═══════════╪═══════════╪══════════╪═════╪═════════╪════════════╪═══════════╪════════════╡
│ Total      ┆ 2474406.